#### Import everything you need

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json

#### Ring a bell!

In [79]:
def inform_exit():
    
    os.system('say "Training process is completed"')

In [80]:
inform_exit()

#### Load Data

In [2]:
def Load_Text_Data(file_path='../trump_tweet_data_archive'):
    """
    Reads the input data.

    :param file_path: (Optional) Position of the txt file in the local system.

    :return: tweets: all input tweets and unique_characters: unique single characters of the input data.
    """
    tweets = []
    for year in range(2009, 2018):
        with open(f'../trump_tweet_data_archive/condensed_{year}.json', ) as condensed:
            data = json.load(condensed)
            for text in range(len(data)):
                text = data[text]['text']
                text += '±'
                tweets.append( text )
    all_chars = ''
    for tweet in tweets:
        all_chars += tweet
    unique_characters = list(set(all_chars))
    return tweets, unique_characters

#### Characters to Index transformation

In [13]:
def Char_to_Ind(chars_list, unique_chars):
    """
    Maps the original characters to integers.

    :param chars_list: The list of characters to be encoded into integers.
    :param unique_chars: The set of unique characters available.

    :return: A list of integers that correspond to the characters.
    """

    encoded_characters = []

    for char in chars_list:
        for index, letter in enumerate(unique_chars):

            if char == letter:

                encoded_characters.append(index)

    return encoded_characters

#### Index to Char transformation

In [14]:
def Ind_to_Char(one_hot_representation, unique_chars_list):
    """
    Maps a list of integers to their corresponding characters,

    :param one_hot_representation: A list of one_hot representations to be transformed to their correspoding characters.
    :param unique_chars_list: The list of unique characters.

    :return: The actual character sequence.
    """

    actual_character_sequence = []

    for i in range(one_hot_representation.shape[1]):

        letter_pos = np.where(one_hot_representation[:,i] == 1.0)[0][0]
        actual_character_sequence.append(unique_chars_list[letter_pos])

    return actual_character_sequence

In [15]:
def create_one_hot_endoding(x, K):
    """
    Creates the one hot encoding representation of an array.


    :param x: The array that we wish to map in an one-hot representation.
    :param K: The number of distinct classes.

    :return: One hot representation of this number.
    """

    x_encoded = np.zeros((K, len(x)))
    for index, elem in enumerate(x):

        x_encoded[elem, index] = 1.0

    return x_encoded

#### Softmax

In [16]:
def softmax(X, theta=1.0, axis=None):
    """
    Softmax over numpy rows and columns, taking care for overflow cases
    Many thanks to https://nolanbconaway.github.io/blog/2017/softmax-numpy
    Usage: Softmax over rows-> axis =0, softmax over columns ->axis =1

    :param X: ND-Array. Probably should be floats.
    :param theta: float parameter, used as a multiplier prior to exponentiation. Default = 1.0
    :param axis (optional): axis to compute values along. Default is the first non-singleton axis.

    :return: An array the same size as X. The result will sum to 1 along the specified axis
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)
    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis=axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis=axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [17]:
def visualize_smoothed_loss(smoothed_loss, display=False, title=None, save_name=None, save_path='../figures/'):
    """
        Visualization and saving the loss of the network.

        :param smoothed_loss: The smooth loss of the RNN network.
        :param display: (Optional) Boolean, set to True for displaying the loss evolution plot.
        :param title: (Optional) Title of the plot.
        :param save_name: (Optional) name of the file to save the plot.
        :param save_path: (Optional) Path of the folder to save the plot in your local computer.

        :return: None

        """

    if title is not None:
        plt.title(title)

    plt.plot(smoothed_loss)

    if save_name is not None:
        if save_path[-1] != '/':
            save_path += '/'
        plt.savefig(save_path + save_name + '.png')

    if display:
        plt.show()

    plt.clf()

### Vanilla Recurrent Neural Network Class

In [19]:
class RNN:
    """
    Recurrent Neural Network object
    """

    def __init__(self, m, K, eta, seq_length, std, epsilon=1e-10):
        """
        Initial setting of the RNN.

        :param m: Dimensionality of the hidden state.
        :param K: Number of unique classes to identify.
        :param eta: The learning rate of the training process.
        :param seq_length: The length of the input sequence.
        :param std: the variance of the normal distribution that initializes the weight matrices.
        :param epsilon: epsilon parameter of ada-grad.
        """

        self.m = m
        self.K = K
        self.eta = eta
        self.seq_length = seq_length
        self.std = std
        self.epsilon = epsilon

    def init_weights(self):
        """
        Initializes the weights and bias matrices
        """

        np.random.seed(400)

        U = np.random.normal(0, self.std, size=(self.m, self.K))
        W = np.random.normal(0, self.std, size=(self.m, self.m))
        V = np.random.normal(0, self.std, size=(self.K, self.m))

        b = np.zeros((self.m, 1))
        c = np.zeros((self.K, 1))

        return [W, U, b, V, c]

    def synthesize_sequence(self, h0, x0, weight_parameters, text_length):
        """
        Synthesizes a sequence of characters under the RNN values.

        :param self: The RNN.
        :param h0: Hidden state at time 0.
        :param x0: First (dummy) input vector of the RNN.
        :param weight_parameters: The weighst and biases of the RNN, which are:
            :param W: Hidden-to-Hidden weight matrix.
            :param U: Input-to-Hidden weight matrix.
            :param b: Bias vector of the hidden layer.
            :param V: Hidden-to-Output weight matrix.
            :param c: Bias vector of the output layer.
        :param text_length: The length of the text you wish to generate

        :return: Synthesized text through.
        """

        W, U, b, V, c = weight_parameters
        Y = np.zeros(shape=(x0.shape[0], text_length))

        alpha = np.dot(W, h0) + np.dot(U, np.expand_dims(x0[:,0], axis=1)) + b
        h = np.tanh(alpha)
        o = np.dot(V, h) + c
        p = softmax(o)

        # Compute the cumulative sum of p and draw a random sample from [0,1)
        cumulative_sum = np.cumsum(p)
        draw_number = np.random.sample()

        # Find the element that corresponds to this random sample
        pos = np.where(cumulative_sum > draw_number)[0][0]

        # Create one-hot representation of the found position
        Y[pos, 0] = 1.0

        h0 = np.copy(h)
        x0 = np.expand_dims(np.copy(Y[:,0]), axis=1)

        for index in range(1, text_length):

            alpha = np.dot(W, h0) + np.dot(U, x0) + b
            h = np.tanh(alpha)
            o = np.dot(V, h) + c
            p = softmax(o)

            # Compute the cumulative sum of p and draw a random sample from [0,1)
            cumulative_sum = np.cumsum(p)
            draw_number = np.random.sample()

            # Find the element that corresponds to this random sample
            pos = np.where(cumulative_sum > draw_number)[0][0]

            # Create one-hot representation of the found position
            Y[pos, index] = 1.0

            h0 = np.copy(h)
            x0 = np.expand_dims(np.copy(Y[:, index]), axis=1)

        return Y

    def ComputeLoss(self, input_sequence, Y, weight_parameters):
        """
        Computes the cross-entropy loss of the RNN.

        :param input_sequence: The input sequence.
        :param weight_parameters: Weights and matrices of the RNN, which in particularly are:
            :param W: Hidden-to-Hidden weight matrix.
            :param U: Input-to-Hidden weight matrix.
            :param b: Bias vector of the hidden layer.
            :param V: Hidden-to-Output weight matrix.
            :param c: Bias vector of the output layer.

        :return: Cross entropy loss (divergence between the predictions and the true output)
        """

        p = self.ForwardPass(input_sequence, weight_parameters)[2]

        cross_entropy_loss = -np.log(np.diag(np.dot(Y.T, p))).sum()

        return cross_entropy_loss

    def ForwardPass(self, input_sequence, weight_parameters, h0 = None):
        """
        Evaluates the predictions that the RNN does in an input character sequence.

        :param input_sequence: The one-hot representation of the input sequence.
        :param weight_parameters: The weights and biases of the network, which are:
            :param W: Hidden-to-Hidden weight matrix.
            :param U: Input-to-Hidden weight matrix.
            :param b: Bias vector of the hidden layer.
            :param V: Hidden-to-Output weight matrix.
            :param c: Bias vector of the output layer.
        :param h0: Initial hidden state value.

        :return: The predicted character sequence based on the input one.
        """

        W, U, b, V, c = weight_parameters

        p = np.zeros(input_sequence.shape)
        if h0 is None:
            h_list = [np.zeros((self.m, 1))]
        else:
            h_list = [h0]

        a_list = []

        for index in range(0, input_sequence.shape[1]):

            alpha = np.dot(W, h_list[-1]) + np.dot(U, np.expand_dims(input_sequence[:,index], axis=1)) + b
            a_list.append(alpha)
            h = np.tanh(alpha)
            h_list.append(h)
            o = np.dot(V, h) + c
            p[:, index] = softmax(o).reshape(p.shape[0],)

        return a_list, h_list[1:], p

    def BackwardPass(self, x, Y, p, W, V, a, h, with_clipping=True):
        """
        Computes the gradient updates of the network's weight and bias matrices based on the divergence between the
        prediction and the true output.

        :param x: Input data to the network.
        :param p: Predictions of the network.
        :param Y: One-hot representation of the correct sequence.
        :param W: Hidden-to-Hidden weight matrix.
        :param V: Hidden-to-Output weight matrix.
        :param a: Hidden states before non-linearity.
        :param h: Hidden states of the network at each time step.
        :param with_clipping: (Optional) Set to False for not clipping in he gradients

        :return:  Gradient updates.
        """

        # grad_W, grad_U, grad_b, grad_V, grad_c = \
        #     np.zeros(W.shape), np.zeros((W.shape[0], x.shape[1])), np.zeros((W.shape[0], 1)), np.zeros(V), np.zeros((x.shape[0], 1))

        # Computing the gradients for the last time step

        grad_c = np.expand_dims((p[:, x.shape[1]- 1] - Y[:, x.shape[1]- 1]).T, axis=1)
        grad_V = np.dot(grad_c, h[-1].T)

        grad_h = np.dot(V.T, grad_c)

        grad_b = np.expand_dims(np.dot(grad_h.reshape((grad_h.shape[0],)), np.diag(1 - np.tanh(a[-1].reshape((a[-1].shape[0],))) ** 2)), axis=1)

        grad_W = np.dot(grad_b, h[-2].T)
        grad_U = np.dot(grad_b, np.expand_dims(x[:,-1], axis=0))

        grad_a = grad_b

        for time_step in reversed(range(x.shape[1]- 1)):

            grad_o = np.expand_dims((p[:, time_step] - Y[:, time_step]).T, axis=1)
            grad_V += np.dot(grad_o, np.transpose(h[time_step]))
            grad_c += grad_o

            grad_h = np.dot(V.T, grad_o) + np.dot(grad_a.T, W).T

            grad_a = np.expand_dims(np.dot(grad_h.reshape((grad_h.shape[0],)), np.diag(1 - np.tanh(a[time_step].reshape((a[time_step].shape[0],))) ** 2)), axis=1)

            grad_W += np.dot(grad_a, h[time_step-1].T)
            grad_U += np.dot(grad_a, np.expand_dims(x[:,time_step], axis=1).T)

            grad_b += grad_a

        gradients = [grad_W, grad_U, grad_b, grad_V, grad_c]

        if with_clipping:

            for index, elem in enumerate(gradients):

                gradients[index] = np.maximum(-5, np.minimum(elem, 5))

        return gradients

    def initialize_ada_grad(self, weight_parameters):
        """
        Initializes the ada_grads of the weights parameters.

        :param weight_parameters: The weights and biases of the RNN

        :return: Initialized ada-grad parameters.
        """

        ada_grads = []

        for elem in weight_parameters:

            ada_grads.append(np.zeros(shape=elem.shape))

        return ada_grads

    def ada_grad_update(self, weight_parameters, ada_grads, gradients, eta):
        """
        Conducts one update step of the ada-grants and weights parameters based on the currently estimated gradient updates.

        :param weight_parameters: Weights and biases of the RNN network.
        :param ada_grads: Ada grad parameters.
        :param gradients: Gradient updates of a training step.
        :param eta: Learning rate of the training process.

        :return: Updated weight and ada_grad parameters.
        """

        # Update ada-grads
        for index, elem in enumerate(ada_grads):

            elem += gradients[index] ** 2

        for index, weight_elem in enumerate(weight_parameters):

            weight_elem -= eta * gradients[index] / np.sqrt(ada_grads[index] + self.epsilon)

        return weight_parameters, ada_grads

    def fit(self, tweets, epoches, unique_characters, verbose=True, with_break=False):
        """
        Comnducts the training pprocess of the RNN nad estimates the model.

        :param tweets: Tweets of Donald trump.
        :param epoches: Number of training epochs.
        :param unique_characters: The unique characters that can be generated from the training process.
        :param verbose: (Optional) Set to False if you do not wish to print synthesized texts during training.
        :param with_break: (Optional) Set to True if you want to break after 100 000 update steps.

        :return: The trained model.
        """

        weight_parameters = self.init_weights()
        gradient_object = Gradients(self)

        # Number of distinct training sequences per epoch

        ada_grads = self.initialize_ada_grad(weight_parameters)

        start = True

        tweet_factor =0

        current_update_step = 0

        for epoch in range(epoches):

            for tweet in tweets:

                integer_encoding = Char_to_Ind(tweet, unique_characters)
                X = create_one_hot_endoding(integer_encoding, len(unique_characters))
                Y = create_one_hot_endoding(integer_encoding, len(unique_characters))

                hprev = np.zeros(shape=(self.m, 1))

                training_sequences_per_epoch = X.shape[1] - self.seq_length

                for e in range(training_sequences_per_epoch):

                    current_update_step += 1

                    x = X[:, e:e + self.seq_length]
                    y = Y[:, e + 1:e + self.seq_length + 1]

                    gradient_updates, hprev = gradient_object.ComputeGradients(x, y, weight_parameters, hprev)

                    weight_parameters, ada_grads = self.ada_grad_update(weight_parameters, ada_grads, gradient_updates, eta=self.eta)

                    if epoch == 0 and e == 0 and start:
                        smooth_loss_evolution = [self.ComputeLoss(x, y, weight_parameters)]
                        minimum_loss = smooth_loss_evolution[0]
                        start = False

                    else:
                        current_loss = 0.999 * smooth_loss_evolution[-1] + 0.001 * self.ComputeLoss(x, y, weight_parameters)
                        smooth_loss_evolution.append(current_loss)
                        if current_loss < minimum_loss:
                            best_weights = weight_parameters
                        best_h_prev = hprev

                    if verbose:

                        if len(smooth_loss_evolution) % 1000 == 0 and len(smooth_loss_evolution) > 0:
                            print('---------------------------------------------------------')
                            print(f'Smooth loss at update step no.{current_update_step}: {smooth_loss_evolution[-1]}')

                            # Also generate synthesized text if 500 updates steps have been conducted
                            if len(smooth_loss_evolution) % 10000 == 0 and len(smooth_loss_evolution) > 0:
                                synthesized_text = Ind_to_Char(
                                    self.synthesize_sequence(h0=hprev, x0=X, weight_parameters=weight_parameters,
                                                             text_length=140), unique_characters)
                                print('---------------------------------------------------------')
                                print(f'Synthesized text of update step no.{current_update_step}')
                                print(''.join(synthesized_text))

                    if with_break and current_update_step == 100000:
                        return best_weights, best_h_prev, smooth_loss_evolution


        return best_weights, best_h_prev, smooth_loss_evolution

### Gradients Class

In [12]:
class Gradients:

    def __init__(self, RNN):
        self.RNN = RNN

    def ComputeGradients(self, X, Y, weight_parameters, hprev, with_clipping=True):
        """
        Computes the analytical gradient updates of the network.
        :param X: Input sequence.
        :param Y: True output
        :param weight_parameters: Weights and bias matrices of the network.
        :param hprev: Initial hidden state to be used in the forward and backwward pass of the RNN.
        :param with_clipping: (Optional) Set ot False if you don't wish to apply clipping in the gradients.

        :return: Gradients updates.
        """

        a_list, h_list, p = self.RNN.ForwardPass(X, weight_parameters, h0=hprev)
        gradients = self.RNN.BackwardPass(X, Y, p, weight_parameters[0], weight_parameters[3], a_list, h_list, with_clipping)

        return gradients, h_list[0]

    def ComputeGradsNumSlow(self, X, Y, weight_parameters, h=1e-4):

        from tqdm import tqdm
        all_grads_num = []

        for index, elem in enumerate(weight_parameters):

            grad_elem = np.zeros(elem.shape)
            # h_prev = np.zeros((W.shape[1], 1))

            for i in tqdm(range(elem.shape[0])):
                for j in range(elem.shape[1]):

                    elem_try = np.copy(elem)
                    elem_try[i, j] -= h
                    all_weights_try = weight_parameters.copy()
                    all_weights_try[index] = elem_try
                    c1 = self.RNN.ComputeLoss(X, Y, weight_parameters=all_weights_try)

                    elem_try = np.copy(elem)
                    elem_try[i, j] += h
                    all_weights_try = weight_parameters.copy()
                    all_weights_try[index] = elem_try
                    c2 = self.RNN.ComputeLoss(X, Y, weight_parameters=all_weights_try)

                    grad_elem[i, j] = (c2-c1) / (2*h)

            all_grads_num.append(grad_elem)

        return all_grads_num

    def check_similarity(self, X, Y, weight_parameters, with_cliping = False):
        """
        Computes and compares the analytical and numerical gradients.

        :param X: Input sequence.
        :param Y: True output
        :param weight_parameters: Weights and bias matrices of the network.
        :param with_cliping: (Optional) Set to True to apply clipping in the gradients

        :return: None.
        """

        analytical_gradients, _ = self.ComputeGradients(X, Y, weight_parameters, hprev=np.zeros(shape=(self.RNN.m, 1)))
        numerical_gradients = self.ComputeGradsNumSlow(X, Y, weight_parameters)

        for weight_index in range(len(analytical_gradients)):
            print('-----------------')
            print(f'Weight parameter no. {weight_index+1}:')

            weight_abs = np.abs(analytical_gradients[weight_index] - numerical_gradients[weight_index])

            weight_nominator = np.average(weight_abs)

            grad_weight_abs = np.absolute(analytical_gradients[weight_index])
            grad_weight_num_abs = np.absolute(numerical_gradients[weight_index])

            sum_weight = grad_weight_abs + grad_weight_num_abs

            print(f'Deviation between analytical and numerical gradients: {weight_nominator / np.amax(sum_weight)}')

## EXERCISES

#### Simulate Donald Trump!

In [5]:
tweets, unique_characters = Load_Text_Data()

In [7]:
rnn = RNN(K=len(unique_characters), m=100, eta=0.01, std=0.01, seq_length=25)

In [8]:
weight_parameters = rnn.init_weights()

In [20]:
weights, h_prev, smooth_loss_evolution = rnn.fit(tweets=tweets, unique_characters=unique_characters, epoches=1, verbose=True, with_break=False)

---------------------------------------------------------
Smooth loss at update step no.1000: 101.65821793682916
---------------------------------------------------------
Smooth loss at update step no.2000: 83.60623295615724
---------------------------------------------------------
Smooth loss at update step no.3000: 77.33672286638323
---------------------------------------------------------
Smooth loss at update step no.4000: 70.99448524550809
---------------------------------------------------------
Smooth loss at update step no.5000: 68.6039491957732
---------------------------------------------------------
Smooth loss at update step no.6000: 69.53847317306708
---------------------------------------------------------
Smooth loss at update step no.7000: 71.0078549429974
---------------------------------------------------------
Smooth loss at update step no.8000: 69.10497238877389
---------------------------------------------------------
Smooth loss at update step no.9000: 70.79062004

---------------------------------------------------------
Smooth loss at update step no.61000: 62.42562884236478
---------------------------------------------------------
Smooth loss at update step no.62000: 62.986215113010374
---------------------------------------------------------
Smooth loss at update step no.63000: 63.05061772702974
---------------------------------------------------------
Smooth loss at update step no.64000: 64.54287652682683
---------------------------------------------------------
Smooth loss at update step no.65000: 62.62068107205167
---------------------------------------------------------
Smooth loss at update step no.66000: 62.4755670272317
---------------------------------------------------------
Smooth loss at update step no.67000: 62.63272819030418
---------------------------------------------------------
Smooth loss at update step no.68000: 62.33852525037096
---------------------------------------------------------
Smooth loss at update step no.69000: 6

---------------------------------------------------------
Smooth loss at update step no.120000: 62.22145448115846
---------------------------------------------------------
Synthesized text of update step no.120000
d, s en wasterafd shebise  whit ay and taned .re it--U f, to jec, hnen cory Cisist penene s oa detivet Cagst Sasp bus ; alill ceiling codicep that "" it -vave step reaf’r "- Yer bacustoy s ikcon the y
---------------------------------------------------------
Smooth loss at update step no.121000: 61.721964078762134
---------------------------------------------------------
Smooth loss at update step no.122000: 60.22221587439734
---------------------------------------------------------
Smooth loss at update step no.123000: 60.3906826622626
---------------------------------------------------------
Smooth loss at update step no.124000: 62.29283293352666
---------------------------------------------------------
Smooth loss at update step no.125000: 61.75036198705663
---------------

---------------------------------------------------------
Smooth loss at update step no.176000: 57.55996129440205
---------------------------------------------------------
Smooth loss at update step no.177000: 57.368879250481235
---------------------------------------------------------
Smooth loss at update step no.178000: 59.000915602817614
---------------------------------------------------------
Smooth loss at update step no.179000: 59.39988702494754
---------------------------------------------------------
Smooth loss at update step no.180000: 58.71903808744569
---------------------------------------------------------
Synthesized text of update step no.180000
bivith " preLfrestn treba't on gat comfan uroce thkin hite farevese for wond--BTrmipy nce ratief Gon Lor the Mattiig Brith. Crentirecoust gom @bacave jont tomilhat wobive the eve to eveat Sienty my wi
---------------------------------------------------------
Smooth loss at update step no.181000: 59.549029635854374
------------

---------------------------------------------------------
Smooth loss at update step no.232000: 59.72522201100006
---------------------------------------------------------
Smooth loss at update step no.233000: 59.93696442790613
---------------------------------------------------------
Smooth loss at update step no.234000: 58.01903459018508
---------------------------------------------------------
Smooth loss at update step no.235000: 58.54396756268296
---------------------------------------------------------
Smooth loss at update step no.236000: 57.19782508492645
---------------------------------------------------------
Smooth loss at update step no.237000: 58.27709798964128
---------------------------------------------------------
Smooth loss at update step no.238000: 57.653284045062605
---------------------------------------------------------
Smooth loss at update step no.239000: 58.846783853367945
---------------------------------------------------------
Smooth loss at update step n

...

---------------------------------------------------------
Smooth loss at update step no.2582000: 56.91815364238042
---------------------------------------------------------
Smooth loss at update step no.2583000: 56.28429441738173
---------------------------------------------------------
Smooth loss at update step no.2584000: 56.82170452135097
---------------------------------------------------------
Smooth loss at update step no.2585000: 54.419790829396796
---------------------------------------------------------
Smooth loss at update step no.2586000: 54.05989590164312
---------------------------------------------------------
Smooth loss at update step no.2587000: 55.43043889753945
---------------------------------------------------------
Smooth loss at update step no.2588000: 54.799047330254595
---------------------------------------------------------
Smooth loss at update step no.2589000: 56.18772763800453
---------------------------------------------------------
Smooth loss at updat

---------------------------------------------------------
Smooth loss at update step no.2641000: 53.864668380693836
---------------------------------------------------------
Smooth loss at update step no.2642000: 52.78202287105488
---------------------------------------------------------
Smooth loss at update step no.2643000: 53.79070894773499
---------------------------------------------------------
Smooth loss at update step no.2644000: 59.067026753434455
---------------------------------------------------------
Smooth loss at update step no.2645000: 59.43597420980071
---------------------------------------------------------
Smooth loss at update step no.2646000: 56.83241558553247
---------------------------------------------------------
Smooth loss at update step no.2647000: 55.73833347251308
---------------------------------------------------------
Smooth loss at update step no.2648000: 55.16345352657952
---------------------------------------------------------
Smooth loss at updat

---------------------------------------------------------
Smooth loss at update step no.2700000: 55.50474690783533
---------------------------------------------------------
Synthesized text of update step no.2700000
p, and stop ane anl not hay uply to shenst of the sparing ord N.C OngactHas Peep, bo wires hopench Dession a losed sfock ow jug ape it Birlectormale word and the alazook in WOT NE🇺S &amp; #Yook of Son
---------------------------------------------------------
Smooth loss at update step no.2701000: 54.88752991139762
---------------------------------------------------------
Smooth loss at update step no.2702000: 54.91270677065813
---------------------------------------------------------
Smooth loss at update step no.2703000: 54.070067289059146
---------------------------------------------------------
Smooth loss at update step no.2704000: 56.81554277360002
---------------------------------------------------------
Smooth loss at update step no.2705000: 55.81294265209089
-------

---------------------------------------------------------
Smooth loss at update step no.2756000: 54.32215900351968
---------------------------------------------------------
Smooth loss at update step no.2757000: 54.451113525372335
---------------------------------------------------------
Smooth loss at update step no.2758000: 54.381296969215654
---------------------------------------------------------
Smooth loss at update step no.2759000: 57.12349462198291
---------------------------------------------------------
Smooth loss at update step no.2760000: 54.30042749724652
---------------------------------------------------------
Synthesized text of update step no.2760000
 hemes seaply mege to Kheritads a cote the kergaryugruines weronall. https://t.co/Qo3c7LS96しhY htredy @IMkiatemonccr ploken's toveradim stonarte hand pporices...... https://t.co/HL🇺KZSThB9vy Call Clec
---------------------------------------------------------
Smooth loss at update step no.2761000: 52.52641413800963
------

---------------------------------------------------------
Smooth loss at update step no.2812000: 53.20959157978436
---------------------------------------------------------
Smooth loss at update step no.2813000: 50.63930072142303
---------------------------------------------------------
Smooth loss at update step no.2814000: 51.234230105074936
---------------------------------------------------------
Smooth loss at update step no.2815000: 53.99129541248784
---------------------------------------------------------
Smooth loss at update step no.2816000: 52.90200746220638
---------------------------------------------------------
Smooth loss at update step no.2817000: 54.209283865685535
---------------------------------------------------------
Smooth loss at update step no.2818000: 53.691599458433345
---------------------------------------------------------
Smooth loss at update step no.2819000: 53.834375484630534
---------------------------------------------------------
Smooth loss at upd